In [1]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
import re

/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
df = pd.read_excel('./summer.xls')
df_train = df[:15000]
df_test = df[15000:]

In [3]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [5]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, \
    SimpleRNN, LSTM, GRU, Masking, LSTM, GRU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [7]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values

In [8]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [9]:
y_train = df_train['Rating'].values
y_test = df_test['Rating'].values

#### RNN

In [10]:
rnn = Sequential()

rnn.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
rnn.add(Masking(mask_value=0.0))

rnn.add(SimpleRNN(64))
rnn.add(Dense(64, activation='relu'))
rnn.add(Dropout(0.5))
rnn.add(Dense(1, activation='sigmoid'))

rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
def eval(model):
    early_stopping=EarlyStopping(monitor='val_loss')  

    history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
    
    score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
    print('\n')
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [12]:
eval(rnn)

Epoch 1/10
27/27 [==============================] - 3s 113ms/step - loss: -11.9315 - accuracy: 0.0864 - val_loss: -25.9587 - val_accuracy: 0.1207
Epoch 2/10
27/27 [==============================] - 3s 122ms/step - loss: -44.6909 - accuracy: 0.0882 - val_loss: -63.1289 - val_accuracy: 0.1207
Epoch 3/10
27/27 [==============================] - 3s 118ms/step - loss: -89.8441 - accuracy: 0.0882 - val_loss: -112.9749 - val_accuracy: 0.1207
Epoch 4/10
27/27 [==============================] - 3s 114ms/step - loss: -149.1974 - accuracy: 0.0882 - val_loss: -177.2270 - val_accuracy: 0.1207
Epoch 5/10
27/27 [==============================] - 3s 104ms/step - loss: -224.0366 - accuracy: 0.0882 - val_loss: -256.6948 - val_accuracy: 0.1207
Epoch 6/10
27/27 [==============================] - 3s 98ms/step - loss: -315.9622 - accuracy: 0.0882 - val_loss: -352.0982 - val_accuracy: 0.1207
Epoch 7/10
27/27 [==============================] - 3s 99ms/step - loss: -424.6941 - accuracy: 0.0882 - val_loss: -463

#### LSTM

In [13]:
lstm = Sequential()

lstm.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
lstm.add(Masking(mask_value=0.0))

lstm.add(LSTM(10))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dropout(0.5))
lstm.add(Dense(1, activation='sigmoid'))

lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
eval(lstm)

Epoch 1/10
27/27 [==============================] - 3s 115ms/step - loss: -0.3686 - accuracy: 0.0860 - val_loss: -2.4766 - val_accuracy: 0.1207
Epoch 2/10
27/27 [==============================] - 2s 75ms/step - loss: -7.2144 - accuracy: 0.0882 - val_loss: -12.5423 - val_accuracy: 0.1207
Epoch 3/10
27/27 [==============================] - 2s 75ms/step - loss: -19.8413 - accuracy: 0.0882 - val_loss: -27.2232 - val_accuracy: 0.1207
Epoch 4/10
27/27 [==============================] - 2s 73ms/step - loss: -38.9613 - accuracy: 0.0882 - val_loss: -49.3673 - val_accuracy: 0.1207
Epoch 5/10
27/27 [==============================] - 2s 74ms/step - loss: -65.7146 - accuracy: 0.0882 - val_loss: -77.7983 - val_accuracy: 0.1207
Epoch 6/10
27/27 [==============================] - 2s 76ms/step - loss: -98.0237 - accuracy: 0.0882 - val_loss: -110.3944 - val_accuracy: 0.1207
Epoch 7/10
27/27 [==============================] - 2s 79ms/step - loss: -133.9345 - accuracy: 0.0882 - val_loss: -146.5698 - val_a

#### GRU

In [15]:
gru = Sequential()

gru.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
gru.add(Masking(mask_value=0.0))

gru.add(GRU(4))
gru.add(Dense(64, activation='relu'))
gru.add(Dropout(0.5))
gru.add(Dense(1, activation='sigmoid'))

gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
eval(gru)

Epoch 1/10
27/27 [==============================] - 3s 108ms/step - loss: 0.1725 - accuracy: 0.0864 - val_loss: -0.5444 - val_accuracy: 0.1207
Epoch 2/10
27/27 [==============================] - 2s 64ms/step - loss: -1.9349 - accuracy: 0.0882 - val_loss: -3.7594 - val_accuracy: 0.1207
Epoch 3/10
27/27 [==============================] - 2s 67ms/step - loss: -7.3939 - accuracy: 0.0882 - val_loss: -11.5551 - val_accuracy: 0.1207
Epoch 4/10
27/27 [==============================] - 2s 66ms/step - loss: -18.6751 - accuracy: 0.0882 - val_loss: -24.9147 - val_accuracy: 0.1207
Epoch 5/10
27/27 [==============================] - 2s 69ms/step - loss: -34.7091 - accuracy: 0.0882 - val_loss: -41.2709 - val_accuracy: 0.1207
Epoch 6/10
27/27 [==============================] - 2s 66ms/step - loss: -53.3430 - accuracy: 0.0882 - val_loss: -59.5909 - val_accuracy: 0.1207
Epoch 7/10
27/27 [==============================] - 2s 71ms/step - loss: -73.6197 - accuracy: 0.0882 - val_loss: -79.6240 - val_accurac

Одинаково плохо на всех моделях. Не так подготовил данные?